# Aluminium–Lutetium (Al–Lu) Phase Diagram and Equilibrium Analysis

This notebook uses PyCalphad to:
- Load the provided `Al-LU.tdb` database
- Generate the Al–Lu binary phase diagram
- Identify/annotate invariant reactions (eutectic/peritectic candidates)
- Compute equilibrium phase fractions for Al-30Lu vs temperature (°C)
- Provide guidance for analysis and interpretation

Prerequisites: `pip install -r requirements.txt`



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, variables as v, equilibrium
from pycalphad.plot.binary import binplot

# Load database and define components
DB_PATH = 'Al-LU.tdb'
dbf = Database(DB_PATH)
components = ['AL', 'LU', 'VA']

# Use all phases from DB for accuracy (can be narrowed if too slow)
all_phases = sorted(dbf.phases.keys())
# Optionally filter out gas if present (most metallic DBs won't have 'GAS')
phases = [p for p in all_phases if p.upper() != 'GAS']

print(f"Loaded DB: {DB_PATH}")
print(f"Number of phases considered: {len(phases)}")
print("Example phases:", phases[:10])



In [ ]:
# Binary phase diagram with annotations
fig, ax = plt.subplots(figsize=(9, 7))
binplot(dbf, components, phases, x=v.X('LU'), T=(500, 3000, 141), P=101325, ax=ax)
ax.set_xlim(0, 1)
ax.set_xlabel('X(LU)')
ax.set_ylabel('Temperature (K)')
ax.set_title('Al–Lu Binary Phase Diagram')

# --- Detect and annotate invariant reactions (heuristic) ---
from typing import List, Tuple

def _stable_phase_set_at(dbf: Database, comps: List[str], phases: List[str], x_lu: float, temp_k: float,
                         threshold: float = 1e-6) -> set:
    eqt = equilibrium(dbf, comps, phases, {v.X('LU'): x_lu, v.T: temp_k, v.P: 101325})
    Wt = eqt['W'].fillna(0)
    if 'vertex' in Wt.dims:
        Wt = Wt.max('vertex')
    if 'component' in Wt.dims:
        Wt = Wt.sum('component')
    phase_names = [str(p) for p in Wt['phase'].values.tolist()]
    stables = []
    for p in phase_names:
        wval = float(np.asarray(Wt.sel(phase=p)))
        if wval > threshold:
            stables.append(p)
    return set(stables)

def _classify_invariant(dbf: Database, comps: List[str], phases: List[str], x_lu: float, temp_k: float,
                        delta_k: float = 2.0) -> str:
    above = _stable_phase_set_at(dbf, comps, phases, x_lu, temp_k + delta_k)
    below = _stable_phase_set_at(dbf, comps, phases, x_lu, temp_k - delta_k)
    # Eutectic: above ~ L + S, below ~ S + S (no L)
    if ('LIQUID' in above and len(above) == 2) and ('LIQUID' not in below and len(below) >= 2):
        return 'eutectic'
    # Peritectic: above ~ L + S1, below ~ S2 (single)
    if ('LIQUID' in above and len(above) == 2) and ('LIQUID' not in below and len(below) == 1):
        return 'peritectic'
    return 'invariant'

# Coarse scan to find likely invariant points where L + two solids coexist
xs = np.linspace(0, 1, 81)
ts = np.linspace(500, 3000, 161)
eq = equilibrium(dbf, components, phases, {v.X('LU'): xs, v.T: ts, v.P: 101325})
W = eq['W'].fillna(0)
if 'vertex' in W.dims:
    W = W.max('vertex')
if 'component' in W.dims:
    W = W.sum('component')

phase_names = [str(p) for p in W['phase'].values.tolist()]
if 'LIQUID' in phase_names:
    stable_counts = (W > 1e-6).sum(dim='phase')
    liq = W.sel(phase='LIQUID') if 'LIQUID' in W['phase'] else 0*W.isel(phase=0)
    invariant_mask = (liq > 1e-6) & (stable_counts >= 3)

    # Identify clusters along temperature lines
    x_coord_name = [c for c in eq.coords if c.startswith('X_')][0]
    X_grid = np.asarray(eq.coords[x_coord_name])
    T_grid = np.asarray(eq.coords['T'])

    mask_vals = np.asarray(invariant_mask)
    dims = list(invariant_mask.dims)
    xi = dims.index(x_coord_name)
    ti = dims.index('T')

    points = np.argwhere(mask_vals)
    marked = set()
    for t_index in np.unique(points[:, ti]):
        xs_at_t = points[points[:, ti] == t_index][:, xi]
        if xs_at_t.size == 0:
            continue
        x_index = int(np.round(xs_at_t.mean()))
        Xc = float(X_grid[x_index])
        Tc = float(T_grid[t_index])
        key = (round(Xc, 4), round(Tc, 2))
        if key in marked:
            continue
        marked.add(key)
        itype = _classify_invariant(dbf, components, phases, Xc, Tc)
        if itype == 'eutectic':
            ax.plot(Xc, Tc, 'o', ms=6, color='cyan', mec='k', label='_eutectic')
            ax.annotate('E', (Xc, Tc), xytext=(5, 5), textcoords='offset points', fontsize=8, color='cyan')
        elif itype == 'peritectic':
            ax.plot(Xc, Tc, 's', ms=6, color='magenta', mec='k', label='_peritectic')
            ax.annotate('P', (Xc, Tc), xytext=(5, 5), textcoords='offset points', fontsize=8, color='magenta')
        else:
            # fallback marker for other invariants
            ax.plot(Xc, Tc, 'd', ms=5, color='gray', mec='k', label='_invariant')

ax.legend(handles=[
    plt.Line2D([0], [0], marker='o', color='w', label='Eutectic', markerfacecolor='cyan', markeredgecolor='k', markersize=7),
    plt.Line2D([0], [0], marker='s', color='w', label='Peritectic', markerfacecolor='magenta', markeredgecolor='k', markersize=7)
], loc='best', frameon=True)

plt.show()



In [ ]:
# Equilibrium at fixed composition: X(LU) = 0.30
X_target = 0.30
T_range = (400, 2000, 161)  # K

# Compute equilibrium
EQ30 = equilibrium(dbf, components, phases, {v.X('LU'): X_target, v.T: T_range, v.P: 101325})

W = EQ30['W'].fillna(0)
if 'vertex' in W.dims:
    W = W.max('vertex')
if 'component' in W.dims:
    W = W.sum('component')

phase_names = [str(p) for p in W['phase'].values.tolist()]
T_vals_K = np.asarray(EQ30.coords['T'])
T_vals_C = T_vals_K - 273.15

# Collect phase fractions across T
y_by_phase = {p: np.asarray(W.sel(phase=p)) for p in phase_names}

# Plot phase fractions
fig, ax = plt.subplots(figsize=(9, 6))
for p, y in y_by_phase.items():
    if np.nanmax(y) > 1e-4:
        ax.plot(T_vals_C, y, label=p)

ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Phase fraction')
ax.set_title('Al-30Lu Phase Fractions vs Temperature')
ax.set_xlim(T_vals_C.min(), T_vals_C.max())
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', ncol=2, fontsize=8)

# Identify invariant-like features on this composition path
# Heuristic: where LIQUID is present and total stable phases >= 3 at (approximately) a single T
counts = (W > 1e-6).sum(dim='phase').values
liq_frac = y_by_phase['LIQUID'] if 'LIQUID' in y_by_phase else np.zeros_like(T_vals_K)
candidates = np.where((liq_frac > 1e-6) & (counts >= 3))[0]

annotated_ts = []
for idx in candidates:
    Tc = float(T_vals_K[idx])
    itype = _classify_invariant(dbf, components, phases, X_target, Tc)
    if itype in ('eutectic', 'peritectic'):
        color = 'cyan' if itype == 'eutectic' else 'magenta'
        ax.axvline(T_vals_C[idx], color=color, linestyle='--', alpha=0.5)
        ax.text(T_vals_C[idx], 0.95, 'E' if itype == 'eutectic' else 'P', color=color,
                ha='center', va='top', fontsize=9, bbox=dict(facecolor='white', alpha=0.6, edgecolor=color))
        annotated_ts.append((itype, Tc))

plt.show()

# Print summary of detected transformations on this composition path
if annotated_ts:
    print('Detected invariant transformations for X(LU)=0.30:')
    for itype, Tk in annotated_ts:
        print(f"  - {itype.capitalize()} near T ≈ {Tk:.1f} K ({Tk-273.15:.1f} °C)")
else:
    print('No clear invariant transformation detected along this exact composition path.')



## Analysis and Interpretation (Guidance)

Use the phase diagram and the Al-30Lu phase-fraction plot to answer:

- Solidification path of Al-30Lu:
  - From liquid on cooling, which phase(s) appear first? At which temperature(s)?
  - Do you pass through an invariant reaction annotated as E (eutectic) or P (peritectic)?
  - Which phases increase/decrease (or disappear) as temperature drops?

- Comparison to a simpler isomorphous system (e.g., Cu–Ni):
  - Cu–Ni shows complete mutual solubility in FCC with no invariant reactions in the liquidus/solidus.
  - Al–Lu exhibits multiple intermetallics and invariant reactions (eutectics/peritectics), leading to more complex solidification paths.

- Role of rare-earth (Lu) in phase stability and kinetics:
  - Rare-earth additions often stabilize ordered intermetallic phases with high melting points and narrow homogeneity ranges.
  - These phases can refine microstructures (via eutectics), strengthen alloys, and influence transformation kinetics (e.g., diffusion-limited peritectics).
  - Processing implications: casting window (liquidus–solidus gap), hot-workability (high-T phase fields), and potential heat-treatment routes.

Cite specific temperatures and phases from your plots (e.g., “Peritectic near ~XXXX K; phases: LIQUID + A → B”).
